In [1]:
import numpy as np

Perceptron


In [141]:
class Perceptron(object):
    def __init__(self, input_size):
        self.weights = np.random.randn(input_size)
        self.bias = np.random.rand(1)

    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoidprime(self,z):
        return np.exp(-z)/((1+np.exp(-z))**2)

    def forward(self, X):
        return self.sigmoid(np.dot(X,self.weights) + self.bias)

    def cost(self, X, Y):
        yhat = self.forward(X)
        return (1/X.shape[0])*np.sum(-Y*np.log(yhat)-(1-Y)*np.log(1-yhat))

    def gradient(self, X, Y):
        yhat = self.forward(X)
        wprime = (1/X.shape[0]) * np.dot((Y-yhat),X)
        bprime = 1/X.shape[0] * np.sum((Y-yhat))
        return wprime, bprime

    def train(self, X, Y):
        cost = self.cost(X,Y)
        previous_cost = cost+1
        wprime, bprime = self.gradient(X,Y)
        while(abs(previous_cost-cost)>=1e-5):
            # print(self.weights, self.bias)
            self.weights+=wprime
            self.bias+=bprime
            previous_cost = cost
            cost = self.cost(X,Y)
            # print(cost)
            wprime, bprime = self.gradient(X,Y)
        # print(self.weights,self.bias)
        return self.weights, self.bias

    def predict(self, X):
        predictions = self.forward(X)
        # print(predictions)
        for i in range(len(predictions)):
            if(predictions[i]>=0.5):
                predictions[i] = 1
            else:
                predictions[i] = 0
        return predictions

And Gate

In [142]:
And = Perceptron(2)
X = np.array([[1,1],[1,0],[0,1],[0,0]])
Y = np.array([1,0,0,0])
weights,bias = And.train(X,Y)

def AND(X):
    return And.predict(X)

We cant make XOR gate with a perceptron.


XOR Gate with a hidden layer

In [143]:
X = np.array([[1,1],[1,0],[0,1],[0,0]])
Y = np.array([0,1,1,0])

number_of_neurons = 2
hidden_layer = np.array([Perceptron(2),Perceptron(2)])
output_layer = Perceptron(number_of_neurons )

h_Y = np.array([[1, 1, 1, 0],[0, 1, 1, 1]])
hidden_outputs = []

for i in range(len(hidden_layer)):
    perceptron = hidden_layer[i]
    perceptron.train(X,h_Y[i])
    hidden_outputs.append(perceptron.forward(X))

hidden_outputs = np.column_stack(hidden_outputs)
  
output_layer.train(hidden_outputs, Y)
hidden_outputs = np.column_stack([perceptron.forward(X) for perceptron in hidden_layer])

def XOR(X):
    hidden_outputs = np.column_stack([perceptron.forward(X) for perceptron in hidden_layer])
    return output_layer.predict(hidden_outputs)

predictions = XOR(X)
print("Predictions for XOR:")
print(predictions)

Predictions for XOR:
[0. 1. 1. 0.]


Full Adder


In [144]:
Or = Perceptron(2)
X = np.array([[1,1],[1,0],[0,1],[0,0]])
Y = np.array([1,1,1,0])
weights,bias = Or.train(X,Y)

def OR(X):
    return Or.predict(X)

def full_adder(a,b,c):
    s1 = XOR([a,b])
    s2 = XOR([s1[0],c])
    c1 = OR([AND([s1[0],c])[0],AND([a,b])[0]])
    c2 = OR([c1[0],AND([s1[0],c])[0]])
    return s2[0], c2[0]


Ripple Carry Adder


In [145]:
def RCA(a,b):
    c = 0
    sum = []
    sum.extend([0]*len(a))
    for i in range(len(a)-1,-1,-1):
        s,c = full_adder(a[i],b[i],c)
        sum[i] = s
    sum=[c]+sum
    return sum

a = [1,1,1,0]
b = [0,0,0,1]
print("RCA of 1111 and 1111:")
print(RCA(a,b))

RCA of 1111 and 1111:
[0.0, 1.0, 1.0, 1.0, 1.0]
